<img src="../Img/Python_SQL_PI.png" style="width:100%">

<p style="font-size:180%;text-align:center">Лекция 6. Библиотека SQLAlchemy 2.0</p>

<br><br>
21 апреля 2022 года, чт., В4/3501 (4-й Вешняковский проезд, 4), 6-я пара, 17:20 - 18:50 <br>
ПИ20-1, ПИ20-2, ПИ20-3 <br>

25 апреля 2022 года, пн., Щ38/512 (ул. Щербаковская, 38), 1-я пара, 08:30 - 10:00 <br>
ПИ20-4, ПИ20-5, ПИ20-6 <br>

Смирнов Михаил Викторович, доцент Департамента анализа данных и машинного обучения Финансового университета при Правительстве Российской Федерации. mvsmirnov@fa.ru

<br><br>

<p style="font-size:180%;text-align:center">Лекция 7. Библиотека SQLAlchemy 2.0, продолжение</p>

<br><br>
5 мая 2022 года, чт., В4/3501 (4-й Вешняковский проезд, 4), 6-я пара, 17:20 - 18:50 <br>
ПИ20-1, ПИ20-2, ПИ20-3 <br>

23 мая 2022 года, пн., Щ38/512 (ул. Щербаковская, 38), 1-я пара, 08:30 - 10:00 <br>
ПИ20-4, ПИ20-5, ПИ20-6 <br>

Смирнов Михаил Викторович, доцент Департамента анализа данных и машинного обучения Финансового университета при Правительстве Российской Федерации. mvsmirnov@fa.ru

<p style="text-align:center">Москва - 2022</p>

При подготовке материалов учебных занятий использовались источники
- Essential SQLAlchemy: Mapping Python to Databases 2nd Edition. Jason Myers, Rick Copeland. O'Reilly Media, Inc. 2015.
- Официальный сайт СУБД sqlite https://sqlite.org
- Официальный сайт библиотеки sqlalchemy https://www.sqlalchemy.org/


SQLAlchemy реализует два API: Core — ядро и ORM - объектно-реляционное сопоставление.

**SQLAlchemy Core** является базовой архитектурой для SQLAlchemy как «набора инструментов для работы с базами данных». Библиотека предоставляет инструменты для управления подключением к базе данных, взаимодействия с запросами и результатами базы данных, а также для программного использования SQL-выражений.

**SQLAlchemy ORM** основывается на Core, предоставляя дополнительные возможности реляционного сопоставления объектов.
ORM предоставляет дополнительный уровень конфигурации, позволяющий сопоставлять пользовательские классы Python с таблицами базы данных и другими конструкциями, а также механизм, известный как сеанс. Затем он расширяет язык выражений SQL базового уровня, позволяя составлять и вызывать SQL-запросы в терминах пользовательских объектов.

<Table align=left>
<tr><td bgcolor=#0dcaf0></td><td style="text-align:left;font-size:10pt">
Таким цветом слева будут отмечены сведения, принципиально важные для ORM-подхода.
</td></tr></Table>

<br><br>
<a id = Home></a>

<p style="font-size:150%"> Оглавление </p>

1. [Механизм подключения к базе данных](#1)
2. [Транзакции и программный интерфейс приложения](#2)
3. [Метаданные базы данных](#3)
4. [Работа с данными](#4)
5. [Манипуляция данными с помощью ORM](#5)

<p style="font-size:150%"> Проверка версии SQLAlchemy </p>

Для выполнения кода примеров требуется версия SQLAlchemy 1.4.

In [1]:
# Проверка версии SQL Alchemy
import sqlalchemy
sqlalchemy.__version__

'1.4.16'

<p style="font-size:150%"> Версия 1.4 в стиле 2.0 </p>

Планируемая следующая версия SQLAlchemy — это верстия 2.0. В версии 1.4 уже реализован интерфейс в стиле 2.0, и чтобы его использовать, введен параметр `future`.

<a id=1></a>

[К оглавлению](#Home)

# 1. Механизм подключения к базе данных

Начало любого приложения SQLAlchemy — это объект, называемый *Engine* (механизм, движок). Этот объект действует как центральный источник подключения к базе данных, предоставляя как движок, так и пространство хранения, называемое пулом подключений к базой данных. Механизм обычно представляет собой глобальный объект, создаваемый для определенного сервера базы данных и настроенный с использованием строки URL-адреса хоста базы данных или серверной части.

В этой лекции мы будем использовать базу данных SQLite в памяти. Это простой способ протестировать функциональность, не требуя создания новой реальной, либо уже существующей базы данных. Движок создается с помощью функции *create_engine()*. В качестве одного из аргументов используем `future=True`, чтобы начать работу в стиле 2.0.

In [2]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

Параметр в виде строки URL укзаывает на три вещи:
1. Название СУБД, в данном случае это *sqlite*.
2. Программный интерфейс приложения для работы с БД, *pysqlite*.
3. Название и местонахождение БД, в данном случае в памяти.

<p style="font-size:125%">Ленивое подключение</p>

Создание движка *engine* само по себе не приводит к подключению к базе данных до тех пор, пока движок не получит какое-либо задание.

Мы установили значение парметра *future=True*, чтобы использовать стиль версии 2.0:
Мы также установили параметр *echo=True*, чтобы движок журналировал SQL команды. 

<a id=2></a>

[К оглавлению](#Home)

# 2. Транзакции и программный интерфейс приложения

Теперь, когда объект *Engine* готов к работе, изучим базовую работу движка и его основные интерактивные конечные точки, *Connection* и *Result*. Мы дополнительно представим фасад *ORM* для этих объектов, известный как *Session*.

<Table>
<tr><td bgcolor=#0dcaf0></td><td style="text-align:left;font-size:10pt">
    При использовании <i>ORM</i> движок управляется другим объектом, называемым <i>сеансом — Session</i>. Сеанс в SQLAlchemy подчеркивает транзакционный способ выполнения, который в значительной степени идентичен объекту <i>Connection</i>, обсуждаемому ниже. Поэтому, хотя этот подраздел ориентирован на пользователей <i>Core</i>, все концепции здесь по существу также относятся к использованию <i>ORM</i>. В конце этого раздела рассмотрим отличия шаблона выполнения <i>Connection</i> от шаблона <i>Session</i> .
</td></tr></Table>

Поскольку нам еще только предстоит знакомство с языком выражений *SQLAlchemy*, который является основным для этой библиотеки, мы будем здесь использовать функцию, называемую *text()*, которая позволяет нам писать операторы *SQL* как текст. Отметим, что текстовый SQL в повседневном использовании SQLAlchemy является скорее исключением, чем правилом для большинства задач, хотя он всегда остается полностью доступным.

<p style="font-size:150%"> Организация подключения </p>

Единственная цель объекта Engine с точки зрения пользователя — предоставить подключение к базе данных, называемое *Connection*. При работе с *Core* напрямую объект *Connection* обеспечивает взаимодействие с базой данных. Поскольку подключение представляет собой открытый ресурс для базы данных, мы хотим всегда ограничивать область использования этого объекта определенным контекстом, и лучший способ сделать это — использовать форму диспетчера контекста Python, также известную как оператор *with*. Проиллюстрируем выполнение выражения текстовой строки «Hello World» с помощью текстового оператора SQL.

In [3]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2022-05-05 15:17:28,584 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:28,589 INFO sqlalchemy.engine.Engine select 'hello world'
2022-05-05 15:17:28,590 INFO sqlalchemy.engine.Engine [generated in 0.00677s] ()
[('hello world',)]
2022-05-05 15:17:28,593 INFO sqlalchemy.engine.Engine ROLLBACK


В приведенном примере диспетчер контекста обеспечил подключение к базе данных, а также поместил операцию в транзакцию. Использование Python DBAPI по умолчанию означает то, что транзакция всегда выполняется. Когда операция завершается, выдается ROLLBACK для завершения транзакции. Транзакция не фиксируется автоматически. Когда мы хотим зафиксировать изменения, нам нужно вызвать `Connection.commit()`.

<p style="font-size:150%">Завершение изменений</p>

Итак, объект соединения не завершает изменения по умолчанию. Что надо сделать, чтобы завершить изменения? покажем это на примере. Создадим таблицу, вставим в неё данные, а затем завершим изменения с помощью метода `Connection.commit()` внутри той же области, в которой мы с помощью диспетчера контекста создали объект соединения.

In [4]:
# "commit as you go"
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
                 [{"x": 1, "y": 1}, {"x": 2, "y": 4}])
    conn.commit()

2022-05-05 15:17:28,613 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:28,614 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2022-05-05 15:17:28,616 INFO sqlalchemy.engine.Engine [generated in 0.00358s] ()
2022-05-05 15:17:28,619 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-05-05 15:17:28,619 INFO sqlalchemy.engine.Engine [generated in 0.00145s] ((1, 1), (2, 4))
2022-05-05 15:17:28,623 INFO sqlalchemy.engine.Engine COMMIT


Мы создали два оператора SQL, которые обычно являются транзакционными: оператор *CREATE TABLE* и параметризованный оператор *INSERT* (приведенный выше синтаксис параметризации обсуждается несколькими разделами ниже в разделе «Отправка нескольких параметров»). Поскольку мы хотим, чтобы проделанная нами работа была зафиксирована в нашем блоке, мы вызываем метод `Connection.commit()`, который завершает транзакцию. После того, как мы вызовем этот метод внутри блока, мы можем продолжить выполнение других операторов SQL, и, если мы захотим, мы можем снова вызвать `Connection.commit()` для последующих операторов. SQLAlchemy относится к этому стилю как к «фиксации по ходу» выполнения - «commit as you go».

Существует также другой стиль фиксации данных, который заключается в том, что мы можем заранее объявить наш блок «подключения» блоком транзакции. Для этого режима работы мы используем метод `Engine.begin()` для получения соединения, а не метод `Engine.connect()`. Этот метод управляет областью соединения и заключает все действия внутри транзакции с *COMMIT* в случае успеха или *ROLLBACK* в случае возникновения исключения. В дальнейшем этот стиль может именоваться "начать сразу" — *begin once*.

In [5]:
# "begin once"
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}]
    )

2022-05-05 15:17:28,637 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:28,640 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-05-05 15:17:28,643 INFO sqlalchemy.engine.Engine [cached since 0.02384s ago] ((6, 8), (9, 10))
2022-05-05 15:17:28,645 INFO sqlalchemy.engine.Engine COMMIT


<Table>
    <tr><td bgcolor=#adb5bd></td><td style="text-align:left; font-size:10pt">

Что значит BEGIN (implicit)?

В начале блока транзакции журнал показал строку «BEGIN (неявно)». «Неявно» здесь означает, что SQLAlchemy считает, что это начало неявной транзакции DBAPI. Например, вы можете зарегистрировать обработчики событий для перехвата этого события.

Дополнительно о событиях: https://docs.sqlalchemy.org/en/14/core/events.html#core-sql-events
</td></tr></Table>

<p style="font-size:150%"> Основы выполнения SQL-выражений </p>

Мы изучили примеры, в которых операторы SQL выполняются с использованием метода `Connection.execute()` в сочетании с объектом  `text()` и возвратом объекта *Result*. Проиллюстрируем более подробно взаимодействие этих компонентов.

<Table>
<tr><td bgcolor=#0dcaf0></td><td style="text-align:left; font-size:10pt">
Большая часть содержимого этого раздела одинаково хорошо применима к ORM при использовании метода Session.execute(), который работает очень похоже на метод Connection.execute(), включая то, что строки результатов ORM доставляются с использованием объекта <a href="https://docs.sqlalchemy.org/en/14/core/connections.html#sqlalchemy.engine.Result"> Result </a>.
        </td></tr>
</Table>

<p style="font-size:120%">Получение строк</p>

Сначала проиллюстрируем объект *Result*, используя строки, которые мы вставили ранее, выполнив оператор *SELECT* для созданной нами таблицы:

In [6]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2022-05-05 15:17:28,664 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:28,666 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2022-05-05 15:17:28,668 INFO sqlalchemy.engine.Engine [generated in 0.00358s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2022-05-05 15:17:28,671 INFO sqlalchemy.engine.Engine ROLLBACK


Команда «SELECT» выбрала все строки из таблицы. Возвращаемый объект *Result* является итерируемым объектом строк результатов.

*Result* имеет множество методов для выборки и преобразования строк, таких как показанный ранее метод `Result.all()`, который возвращает список всех объектов *Row*. Он также реализует интерфейс итератора `Python`, так что мы можем напрямую перебирать коллекцию объектов *Row*.

Сами объекты *Row* должны действовать как <a href="https://docs.python.org/3/library/collections.html#collections.namedtuple"> именованные кортежи </a> Python. Ниже проиллюстрируем различные способы доступа к строкам.

- Позиционное. Этот стиль заключается в назначении переменных каждому атрибуту строки позиционно:

```
result = conn.execute(text("select x, y from some_table"))
for x, y in result:
    print(x, y)
```

- По индексу. Кортежи — это упорядоченные неизменяемые коллекции, к элементам которых можно обращаться обычным образом по целочисленному индексу

```
with engine.connect() as conn:
    result = conn.execute(text("select x, y from some_table"))
    for row in result:
        x, y = row[0], row[1]
        print(x, y)
```
- По имени атрибута. 
Поскольку это именованные кортежи Python, они имеют динамические имена атрибутов, соответствующие именам каждого столбца. Эти имена обычно являются именами, которые оператор SQL присваивает столбцам в каждой строке.

```
with engine.connect() as conn:
    result = conn.execute(text("select x, y from some_table"))
    for row in result:
        print(f"Row: {row.x} {row.y}")
```

- По сопоставлению. Чтобы получать строки в качестве объектов сопоставления Python, что по сути является версией "для чтения" обычного объекта *dict*, *Result* может быть преобразован в объект *MappingResult* с использованием модификатора `Result.mappings()`. Это объект результата, который дает подобные словарю объекты *RowMapping*, а не объекты *Row*:

```
with engine.connect() as conn:
    result = conn.execute(text("select x, y from some_table"))
    for dict_row in result.mappings():
        x, y = dict_row['x'], dict_row['y']
        print(x,y)
```

<p style="font-size:150%">Отправка параметров</p>

Операторы SQL обычно сопровождаются данными, которые должны быть переданы вместе с самим оператором, как мы видели ранее в примере вставки *INSERT*. Таким образом, метод `Connection.execute()` также принимает параметры, которые называются связанными параметрами. Элементарным примером может быть то, что мы хотим ограничить оператор «SELECT» только строками, которые соответствуют определенным критериям, например, строками, в которых значение «y» превышает определенное значение, переданное функции.

Чтобы добиться этого, мы добавляем в выражение условие *WHERE* новый параметр с именем «y»; функция `text()` принимает его, используя формат двоеточия «:y». Фактическое значение «:y» затем передается в качестве второго аргумента функции `Connection.execute()` в виде словаря:

In [7]:
with engine.connect() as conn:
    result = conn.execute(
        text("SELECT x, y FROM some_table WHERE y > :y"),
        {"y": 2}
    )
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-05-05 15:17:28,695 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:28,697 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2022-05-05 15:17:28,699 INFO sqlalchemy.engine.Engine [generated in 0.00346s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2022-05-05 15:17:28,703 INFO sqlalchemy.engine.Engine ROLLBACK


В журналированном выводе SQL мы видим, что связанный параметр `:y` преобразован в вопросительный знак, когда он был отправлен в базу данных SQLite. Это связано с тем, что драйвер базы данных SQLite использует формат передачи параметров в стиле вопросительного знака, который является форматом, разрешенным спецификацией DBAPI.

<Table><tr><td bgcolor=#adb5bd></td><td style="text-align:left; font-size:10pt">
Как упоминалось в начале этого раздела, текстовый SQL не является обычным способом работы с SQLAlchemy. Однако при использовании текстового SQL литеральные значения Python, даже нетекстовые, такие как целые числа или даты, никогда не должны напрямую преобразовываться в строку SQL; всегда следует использовать параметр. Это наиболее известно как способ избежать атак с помощью SQL-инъекций. Однако это также позволяет диалектам SQLAlchemy и/или DBAPI правильно обрабатывать входящие входные данные для серверной части. За исключением случаев использования простого текстового SQL, SQLAlchemy Core Expression API гарантирует, что литеральные значения Python передаются в качестве связанных параметров, где это необходимо.
</td></tr></Table>

<p style="font-size:150%"> Отправка нескольких параметров </p>

В примере с фиксацией изменений мы выполнили оператор INSERT, где оказалось, что мы можем вставить сразу несколько строк в базу данных. Для операторов, которые работают с данными, но не возвращают наборы результатов, а именно операторов DML, таких как «INSERT», мы можем отправить в метод `Connection.execute()` несколько параметров в списке словарей:

In [8]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}]
    )
    conn.commit()

2022-05-05 15:17:28,724 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:28,726 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-05-05 15:17:28,728 INFO sqlalchemy.engine.Engine [cached since 0.1087s ago] ((11, 12), (13, 14))
2022-05-05 15:17:28,730 INFO sqlalchemy.engine.Engine COMMIT


На самом деле, объекты *Connection* используют возможность DBAPI, известную как `cursor.executemany()`. Этот метод выполняет эквивалентную операцию вызова данного оператора SQL для каждого набора параметров в отдельности. DBAPI может оптимизировать эту операцию различными способами, используя подготовленные операторы или в некоторых случаях объединяя наборы параметров в один оператор SQL. Некоторые диалекты SQLAlchemy могут также использовать альтернативные API для этого случая, такие как диалект *psycopg2* для *PostgreSQL*, который в таких случаях использует более производительные API.

<Table align=left>
<tr><td bgcolor=#0dcaf0></td><td style="text-align:left;font-size:10pt">
Интерфейс ORM использует другой подход передачи нескольких параметров операторам INSERT/UPDATE/DELETE.
</td></tr></Table>

<br><p>
<p style="font-size:150%">Связывание параметров с операторами SQL-выражения</p>

Два предыдущих примера иллюстрируют ряд параметров, передаваемых вместе с оператором SQL. Для выполнения выражений с одним параметром, использование параметров в SQLAlchemy на самом деле чаще всего осуществляется путем связывания с оператором, что является основной функцией языка выражений SQL и позволяет создавать запросы, которые могут быть составлены естественным образом. Эта концепция будет обсуждаться более подробно в следующих разделах. Предварительно отметим, что сама конструкция *text()*, являющаяся частью языка выражений SQL, поддерживает эту функцию с помощью метода `TextClause.bindparams()`; это <a href="https://docs.sqlalchemy.org/en/14/glossary.html#term-generative">генеративный</a> метод, который возвращает новую копию конструкции SQL, дополненную значениями параметров, которые мы хотим передать:

In [9]:
stmt = text("SELECT x, y FROM some_table WHERE (y > :y) AND (x > :x) ORDER BY x, y").bindparams(y=6, x=0)
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-05-05 15:17:28,754 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:28,755 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE (y > ?) AND (x > ?) ORDER BY x, y
2022-05-05 15:17:28,758 INFO sqlalchemy.engine.Engine [generated in 0.00378s] (6, 0)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2022-05-05 15:17:28,761 INFO sqlalchemy.engine.Engine ROLLBACK


Отметим, что в результате связывания параметра с SQL-выражением, метод `Connection.execute()` сформировал как строку SQL, так и отдельный кортеж параметров, в данном случае, из двух элементов.

<p style="font-size:150%">Выполнение сеанса ORM</p>

<Table align=left>
<tr><td bgcolor=#0dcaf0></td><td style="text-align:left;font-size:10pt">
Как упоминалось ранее, большинство приведенных выше шаблонов и примеров также применимы для объектно-реляционного сопоставления ORM.
</td></tr></Table><br><br>

Основным объектом при использовании ORM является сеанс — *Session*. В SQLAlchemy этот объект используется способом, похожим на метод *Connection*, и фактически, когда используется сеанс, он ссылается на внутреннее подключение *Connection*, которое он использует для генерации SQL-выражения.

Сеанс имеет несколько различных шаблонов создания, проиллюстрируем здесь базовый, который отслеживает то, как используется соединение. Создадим сеанс в диспетчере контекста:

In [10]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-05-05 15:17:28,888 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:28,890 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2022-05-05 15:17:28,892 INFO sqlalchemy.engine.Engine [generated in 0.00191s] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2022-05-05 15:17:28,897 INFO sqlalchemy.engine.Engine ROLLBACK


Сравним этот пример с примером в предыдущем параграфе «Связывание параметров с оператором». Мы заменяем вызов `engine.connect() as conn` на `Session(engine) as session`, а затем используем `Session.execute()` точно так же, как и с методом `Connection.execute()`.

Кроме того, как и *Подключение*, *Сеанс* следует шаблону поведения «фиксации по ходу» с использованием метода `Session.commit()`, показанного ниже с использованием  SQL-оператора *UPDATE*.

In [11]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y":11}, {"x": 13, "y": 15}]
    )
    session.commit()

2022-05-05 15:17:28,914 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:28,915 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2022-05-05 15:17:28,918 INFO sqlalchemy.engine.Engine [generated in 0.00175s] ((11, 9), (15, 13))
2022-05-05 15:17:28,920 INFO sqlalchemy.engine.Engine COMMIT


В этом примере мы вызвали оператор UPDATE, используя связанный параметр в стиле «executemany» и заканчивая блок фиксацией по ходу «commit as you go».

<Table align="left">
    <tr><td bgcolor=#0dcaf0></td>
        <td style="text-align:left;font-size:10pt">
Примечание. Сеанс не удерживает подключение Connection после завершения транзакции. Он получает новое соединение от движка, когда в следующий раз требуется выполнение SQL-запроса к базе данных.
        </td></tr>
</Table>

<br><br><p>
У Сеанса, очевидно, значительно больше особенностей, однако понимание того, что метод `Session.execute()` используется так же, как `Connection.execute()`, поможет нам изучить примеры, которые последуют позже.
</p>

Дополнительно: <a href="https://docs.sqlalchemy.org/en/14/orm/session_basics.html#id1">Основы Сеанса</a>

<a id=3></a>

[К оглавлению](#Home)

# 3. Метаданные базы данных

Центральным элементом обоих интерфейсов SQLAlchemy — как Core, так и ORM, является язык выражений SQL, предназначенный для SQL-запросов. Основой для этих запросов являются объекты Python, представляющие такие понятия базы данных, как таблицы и столбцы. Эти объекты вместе известны как <a href="https://docs.sqlalchemy.org/en/14/glossary.html#term-database-metadata">метаданные базы данных</a>.

Наиболее распространенные базовые объекты <a href="https://docs.sqlalchemy.org/en/14/core/metadata.html#sqlalchemy.schema.MetaData"> метаданных </a> базы данных в SQLAlchemy известны как таблица <a href="https://docs.sqlalchemy.org/en/14/core/metadata.html#sqlalchemy.schema.Table"> *Table* </a> и столбец <a href="https://docs.sqlalchemy.org/en/14/core/metadata.html#sqlalchemy.schema.Column"> *Column* </a>. Покажем, как эти объекты используются в интерфейсах *Core* и *ORM*.

<p style="font-size:150%">Создание метаданных таблиц</p>

Когда мы работаем с реляционной базой данных, одной из базовых структур для составления запросов является таблица. В SQLAlchemy «таблица» представлена объектом Python с аналогичным названием <a href="https://docs.sqlalchemy.org/en/14/core/metadata.html#sqlalchemy.schema.Table">*Table*</a>.

Чтобы начать использовать язык выражений SQLAlchemy, нам нужно создать объекты *Table*, представляющие все таблицы базы данных, с которыми мы собираемся работать. Каждая таблица может быть объявлена, то есть мы явно прописываем в исходном коде, как выглядит или **отображается** таблица, что означает, что мы генерируем объект на основе того, что уже присутствует в конкретной базе данных. Эти два подхода также можно комбинировать разными способами.

Независимо от того, будем ли мы объявлять или отображать таблицы БД, мы начинаем с коллекции, известной как объект метаданных. Этот объект, по сути, представляет собой фасад словаря Python, в котором хранится ряд объектов *Table*, привязанных к их строковым именам. Построение объекта Метаданных выглядит так:

In [12]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

Наличие одного объекта *MetaData* для всего приложения является наиболее распространенным случаем. Такой объект представляется переменной уровня модуля в одном месте в приложении, как «модель» или «схема базы данных». В общем случае, в приложении может быть более одного объекта метаданных, но как правило, все объекты таблиц, связанных друг с другом, принадлежат одному объекту метаданных.

Когда у нас есть объект *MetaData*, мы можем объявить объекты Table. Рассмотрим учебную базу данных, в которой таблица пользователей будет представлять пользователей веб-сайта, а таблица адресов — список адресов электронной почты пользователей. Обычно мы присваиваем каждому объекту *Table* переменную, с помощью которой мы будем ссылаться на таблицу в коде приложения.

In [13]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname', String)
)

Заметим, что приведенная выше конструкция таблицы похожа на оператор SQL CREATE TABLE: начинется с имени таблицы, затем перечисляется каждый столбец, указывается имя и тип данных. Объекты, которые мы используем выше:

- Таблица — представляет собой таблицу базы данных и присваивает себя коллекции метаданных.
- Столбец — представляет столбец в таблице базы данных и присваивает себя объекту таблицы. Столбец обычно включает строку имени и тип. Доступ к коллекции столбцов с точки зрения родительской таблицы обычно осуществляется через ассоциативный массив, расположенный в <a href="https://docs.sqlalchemy.org/en/14/core/metadata.html#sqlalchemy.schema.Table.c">Table.c</a>:

In [14]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [15]:
user_table.c.keys()

['id', 'name', 'fullname']

<a href="https://docs.sqlalchemy.org/en/14/core/type_basics.html#sqlalchemy.types.Integer">Integer</a>, <a href="https://docs.sqlalchemy.org/en/14/core/type_basics.html#sqlalchemy.types.String">String</a> — эти классы представляют типы данных SQL и могут быть указаны для столбца с ограничением на размер или без ограничений. Выше мы указали длину «30» для столбца «имя», поэтому мы создали экземпляр String (30). Но для «id» и «fullname» мы размер не указывали.

Дополнительно: <a href="https://docs.sqlalchemy.org/en/14/core/metadata.html">Описание структуры базы данных с помощью метаданных</a>.

В следующем разделе мы проиллюстрируем одну из основных функций объекта таблицы, которая заключается в выполнении команд DDL. Но сначала объявим вторую таблицу.

<p style="font-size:150%">Объявление простых ограничений</p>

Первый столбец в приведенной выше таблице *user_table* включает параметр `Column.primary_key`, который указывает на то, что этот столбец должен быть частью первичного ключа таблицы. Сам первичный ключ представлен конструкцией *PrimaryKeyConstraint*, которую мы можем видеть в атрибуте `Table.primary_key` объекта Table:

In [16]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

Другое ограничение, которое чаще всего объявляется явно — это объект *ForeignKeyConstraint*, соответствующий ограничению внешнего ключа базы данных. Когда мы объявляем таблицы, которые связаны друг с другом, SQLAlchemy использует наличие этих объявлений ограничений внешнего ключа не только для того, чтобы они генерировались в операторах CREATE для базы данных, но и для помощи в построении выражений SQL.

Ограничение *ForeignKeyConstraint*, которое включает только один столбец в целевой таблице, обычно объявляется с использованием сокращенной записи на уровне столбца через объект *ForeignKey*. Ниже мы объявляем вторую таблицу — таблицу адресов, которая будет иметь ограничение внешнего ключа, ссылающееся на пользовательскую таблицу:

In [17]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('user_id', ForeignKey('user_account.id'), nullable=False),
    Column('email_address', String, nullable=False)
)

В приведенной выше таблице также представлен третий тип ограничения, которым в SQL является ограничение «NOT NULL», указанное с помощью параметра `Column.nullable`.

<Table align="left">
    <tr><td bgcolor=#adb5bd></td>
        <td style="text-align:left; font-size:10pt">
Совет.

При использовании объекта ForeignKey в определении столбца мы можем опустить тип данных - он автоматически выводится из связанного столбца; в приведенном выше примере - это тип данных Integer столбца user_account.id.
        </td></tr>
</Table>

Завершим выполнение команд DDL для таблиц пользователей и адресов.

<p style="font-size:150%">Отправка команд DDL в базу данных</p>

Мы построили иерархию объектов для представления двух таблиц базы данных, начиная с корневого объекта *MetaData*, затем два объекта *Table*, каждый из которых содержит набор объектов *Column* и *Constraint*. Эта объектная структура будет в центре большинства последующих операций, которые мы будем выполнять как с *Core*, так и с *ORM*.

Первая полезная вещь, которую мы можем сделать с этой структурой — это отправить операторы DDL CREATE TABLE в нашу базу данных SQLite. У нас уже есть все инструменты, необходимые для этого, а именно: вызов метода `MetaData.create_all()` и отправка ему движка *Engine*, который ссылается на целевую базу данных:

In [18]:
metadata_obj.create_all(engine)

2022-05-05 15:17:29,057 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:29,058 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-05-05 15:17:29,062 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-05 15:17:29,064 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-05-05 15:17:29,065 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-05 15:17:29,067 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-05-05 15:17:29,068 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-05 15:17:29,070 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-05-05 15:17:29,072 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-05 15:17:29,074 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-05-05 15:17:29,076 INFO sqlalchemy.engine.Engine [no key 0.00157s] ()
2022-05-05 15:17:29,079 INFO sqlalchemy.engine.Engine 
C

Процесс создания DDL по умолчанию включает некоторые специфичные для SQLite операторы PRAGMA, которые проверяют существование каждой таблицы перед отправкой CREATE. Полная последовательность шагов включена в пару BEGIN/COMMIT для поддержки транзакционного DDL (SQLite действительно поддерживает транзакционный DDL, однако драйвер базы данных sqlite3 исторически запускает DDL в режиме «автоматической фиксации»).

Процесс создания также заботится об отправке операторов CREATE в правильном порядке: ограничение FOREIGN KEY зависит от существующей пользовательской таблицы, поэтому таблица адресов создается второй. В более сложных сценариях зависимостей ограничения FOREIGN KEY также могут применяться к таблицам постфактум с использованием ALTER.

Объект *MetaData* также имеет метод `MetaData.drop_all()`, который выдает операторы DROP в обратном по отношению к CREATE порядке для удаления элементов схемы.

<p style="font-size:150%">Определение метаданных таблицы с помощью ORM</p>

В этом разделе, посвященном только ORM, будет представлен пример объявления той же структуры базы данных, что и в предыдущем разделе, с использованием парадигмы конфигурации, более ориентированной на ORM. При использовании ORM процесс объявления метаданных таблицы обычно сочетается с процессом объявления сопоставленных классов. Сопоставленный класс — это класс Python, который мы хотели бы создать, который затем будет иметь атрибуты, которые будут связаны со столбцами в таблице базы данных. Хотя существует несколько вариантов того, как это достигается, наиболее распространенный стиль известен как декларативный и позволяет нам одновременно объявлять наши определяемые пользователем классы и метаданные таблицы.

<p style="font-size:150%">Установка реестра</p>

При создании ORM коллекция метаданных содержится в объекте, называемом <a href="https://docs.sqlalchemy.org/en/14/orm/mapping_api.html#sqlalchemy.orm.registry">реестром</a>. Мы создаём реестр вот так:

In [19]:
from sqlalchemy.orm import registry
mapper_registry = registry()

После создания реестр сразу включает в себя объект метаданных, который будет содержать в себе коллекцию таблиц.

In [20]:
mapper_registry.metadata

MetaData()

Вместо того, чтобы объявлять объекты *Table* напрямую, мы теперь будем объявлять их косвенно через директивы, применяемые к нашим отображаемым классам. В наиболее распространенном подходе каждый отображаемый класс происходит от общего базового класса, известного как **декларативная база**. Получим новую декларативную базу из реестра с помощью метода `register.generate_base()`:

```
Base = mapper_registry.generate_base()
```

Совет<br>
Шаги по созданию реестра и класса декларативной базы могут быть объединены с помощью известрной функции *declarative_base()*.

In [21]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()

<p style="font-size:150%">Объявление сопоставленных классов</p>

Приведенный выше базовый объект — это класс Python, который будет служить базовым для классов ORM, которые мы объявляем. Теперь мы можем определить классы ORM для таблиц пользователей и адресов в терминах новых классов *User* и *Address*:

In [22]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship
class User(Base):
    __tablename__ = 'user_account'
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address", back_populates="user")
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('user_account.id'))
    user = relationship("User", back_populates="addresses")
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

Вышеупомянутые два класса теперь являются нашими сопоставленными классами и доступны для использования в операциях запросов, которые будут описаны позже. Но они также включают в себя объекты Table, которые были сгенерированы как часть процесса декларативного сопоставления и эквивалентны тем, которые мы объявили непосредственно в предыдущем разделе Core. Мы можем увидеть эти объекты Table из декларативного сопоставленного класса, используя атрибут `.__table__`:

In [23]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

Объект *Table* был сгенерирован в результате декларативного процесса на основе атрибута `.__tablename__`, определенного для каждого из наших классов, а также посредством использования объектов *Column*, назначенных атрибутам внутри классов. Эти объекты *Column* обычно объявляются без явного поля «имя» внутри конструктора, так как декларативный процесс присвоит имя автоматически на основе имени используемого атрибута.

<p style="font-size:120%">Другие сведения о сопоставленном классе</p>

Обратим внимание на следующие атрибуты:

Классы имеют автоматически сгенерированный метод `__init__()`, которые позволяют осуществлять параметризованное построение объектов. Мы можем также создать собственный метод `__init__()`.  `__init__()` позволяет нам создавать объекты и адреса, передавая имена атрибутов, большинство из которых связаны непосредственно с объектами *Column*, как имена параметров:

In [24]:
sandy = User(name="sandy", fullname="Sandy Cheeks")

метод `__repr__()` необязателен и предназначен лишь для того, чтобы наши пользовательские классы имели описательное строковое представление:

In [25]:
sandy

User(id=None, name='sandy', fullname='Sandy Cheeks')

Отметим, что атрибут *id* автоматически возвращает *None*, а не вызывает *AttributeError*, как этого можно было бы ожидать в Python.

Мы также включили двунаправленное отношение — это еще одна полностью необязательная конструкция,  называемая *relationship()* для обоих классов, которая указывает ORM, что классы *User* и *Address* ссылаются друг на друга в соотношении один ко многим.

<p style="font-size:150%">Отправка DDL в базу данных</p>

Выполнение команды DDL с сопоставленными классами ORM ничем не отличается от аналогичного действия с помощью интерейса Core. Если бы мы хотели выполнить команды DDL для объектов *Table*, которые мы создали как часть наших декларативно сопоставленных классов, мы по-прежнему могли бы использовать `MetaData.create_all()`.

В нашем случае мы уже создали в базе данных таблицы пользователей и адресов. Если бы мы этого еще не сделали, мы могли бы свободно использовать метаданные, связанные с нашим реестром, и декларативный базовый класс ORM, чтобы сделать это, используя `MetaData.create_all()`:

```
# выполнение команды CREATE statements с конкретным реестром ORM
mapper_registry.metadata.create_all(engine)
Base.metadata.create_all(engine)
```

<p style="font-size:150%">Совместное использование интерфейсов Core и ORM Declarative</p>

В качестве альтернативного подхода к процессу сопоставления, показанному ранее в разделе «Объявление сопоставленных классов», мы также можем использовать объекты таблиц, которые мы создали непосредственно в разделе «Создание метаданных таблиц», в сочетании с декларативными сопоставленными классами из сгенерированного `declarative_base()` базового класса.

Такой подход называют гибридным, и он состоит в прямом назначении атрибута `.__table__`, а не в декларативном процессе:

```
mapper_registry = registry()
Base = mapper_registry.generate_base()

class User(Base):
    __table__ = user_table

    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User({self.name!r}, {self.fullname!r})"

class Address(Base):
    __table__ = address_table

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address({self.email_address!r})"
```

Эти два класса эквивалентны классам, созданным в прдыдущем примере раздела "Объявление сопоставленных классов".

<p style="font-size:150%">Отображение таблиц</p>

Отображение таблицы относится к процессу чтения текущего состояния базы данных. В предыдущих разделах мы объявляли табличные объекты в Python, а затем отправляли команды DDL в базу данных. Процесс отображения делает это в обратном порядке.

В качестве примера отражения создадим новый объект таблицы, отображающий таблицу *some_table*, которую мы создали вручную в более ранних разделах этого документа. Используем конструктор таблицы, укажем имя таблицы и объект метаданных, к которым она будет принадлежать. Затем, вместо того, чтобы создавать столбцы и ограничения, передадим в конструктор целевой движок с помощью параметра `Table.autoload_with`:

In [26]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2022-05-05 15:17:29,251 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:29,253 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2022-05-05 15:17:29,255 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-05 15:17:29,259 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-05-05 15:17:29,259 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2022-05-05 15:17:29,267 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2022-05-05 15:17:29,267 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-05 15:17:29,275 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2022-05-05 15:17:29,279 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-05 15:17:29,281 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
20

По завершении этого процесса объект *some_table* содержит информацию о столбцах этой таблицы. Теперь этот объект можно использовать также как и другие таблицы, объявленные явно:

In [27]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

Более подробно об отображении таблиц в <a href="https://docs.sqlalchemy.org/en/14/core/reflection.html">документации</a>.

[К оглавлению](#Home)

<a id=4></a>

# 4. Работа с данными: INSERT, UPDATE, DELETE, SELECT

<p style="font-size:150%; font-weight:bold">4.1. Вставка с помощью Core</p>

Для вставки строк с помощью интерфейса Core используем функцию *insert()*.

In [28]:
from sqlalchemy import insert
stmt = insert(user_table).values(name='tcampion', fullname='Campion Thomas')

Это SQL-выражение можно представить в текстовом виде:

In [29]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


Метод <a href="https://docs.sqlalchemy.org/en/14/core/foundation.html#sqlalchemy.sql.expression.ClauseElement.compile">ClauseElement.compile()</a> возвращает компилированный объект. Если применить его к объекту SQL-выражения, получим

In [30]:
stmt.compile()

Функция *str()* возвращает текстовое представление объекта.

In [31]:
str(stmt.compile())

'INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)'

Свойство *params* показывает значение параметров SQL-выражения.

In [32]:
stmt.compile().params

{'name': 'tcampion', 'fullname': 'Campion Thomas'}

Выполним SQL-выражение.

In [33]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2022-05-05 15:17:29,459 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:29,463 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-05-05 15:17:29,466 INFO sqlalchemy.engine.Engine [generated in 0.00615s] ('tcampion', 'Campion Thomas')
2022-05-05 15:17:29,466 INFO sqlalchemy.engine.Engine COMMIT


В приведенной выше простой форме оператор INSERT возвращает целочисленное значение первичного ключа, которое мы можем получить с помощью метода доступа `CursorResult.inserted_primary_key`:

In [34]:
result.inserted_primary_key

(1,)

Полученное значение является кортежем, так как в общем случае первичный ключ может состоять из более чем одного атрибута. Возвращаемый с помощью *inserted_primary_key* кортеж является именованным, представляет собой объект типа строки *Row*:

In [35]:
row = result.inserted_primary_key
print(row._mapping)

{'id': 1}


В приведенном выше примере использовался метод `Insert.values()` для явного создания предложения VALUES инструкции SQL INSERT. На самом деле у этого метода есть несколько вариантов, которые допускают специальные формы, такие как несколько строк в одном выражении. Обычный способ использования Insert заключается в том, что предложение VALUES генерируется автоматически из параметров, переданных методу Connection.execute(). Вставим еще две строки, чтобы проиллюстрировать это:

In [36]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "nnigrino", "fullname": "Nigrino Niccolo"},
            {"name": "gcaccini", "fullname": "Caccini Giulio"},
            {"name": "talbinoni", "fullname": "Albinoni Thomaso"},
            {"name": "fmilano", "fullname": "Da Milano Francesko"},
            {"name": "gbfontana", "fullname": "Giovanni Battista"}
        ]
    )
    conn.commit()

2022-05-05 15:17:29,536 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:29,537 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-05-05 15:17:29,541 INFO sqlalchemy.engine.Engine [generated in 0.00409s] (('nnigrino', 'Nigrino Niccolo'), ('gcaccini', 'Caccini Giulio'), ('talbinoni', 'Albinoni Thomaso'), ('fmilano', 'Da Milano Francesko'), ('gbfontana', 'Giovanni Battista'))
2022-05-05 15:17:29,542 INFO sqlalchemy.engine.Engine COMMIT


Этот код имеет форму «executemany», ранее использованную нами в подразделе «Отправка нескольких параметров» с помощью конструкции *text()*, однако, теперь нам не потребовалось создавать команду SQL в текстовом виде. Передавая словарь или список словарей в метод `Connection.execute()` в сочетании с конструкцией *Insert*, *Connection* гарантирует, что передаваемые имена столбцов будут автоматически выражены в предложении *VALUES* команды *Insert*.

<p style="font-size:150%">INSERT…FROM SELECT</p>

Конструкция *Insert* может составлять SQL-выражение, в котором команда INSERT получает строки непосредственно из SELECT с помощью метода `Insert.from_select()`:

In [37]:
from sqlalchemy import select

select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com") # Получаем строки таблицы user_account

# Вставляем в таблицу address
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt)

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


In [38]:
with engine.connect() as conn:
    result = conn.execute(insert_stmt)
    conn.commit()

2022-05-05 15:17:29,579 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:29,580 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || ? AS anon_1 
FROM user_account
2022-05-05 15:17:29,582 INFO sqlalchemy.engine.Engine [generated in 0.00362s] ('@aol.com',)
2022-05-05 15:17:29,584 INFO sqlalchemy.engine.Engine COMMIT


<p style="font-size:150%">INSERT…RETURNING</p>

Предложение RETURNING для поддерживаемых серверов БД используется автоматически для извлечения последнего вставленного значения первичного ключа, а также значений по умолчанию для сервера. Однако предложение RETURNING также может быть указано явно с помощью метода `Insert.returning()`, в этом случае объект Result содержит возвращаемые значения:

In [39]:
insert_stmt = insert(address_table).returning(address_table.c.id, address_table.c.email_address)
print(insert_stmt)

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


Предложение RETURNING можно комбинировать с FROM SELECT

In [40]:
select_stmt = select(user_table.c.id, user_table.c.name + "@mail.ru")
select_stmt = select_stmt.where(user_table.c.id > 3)

insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account 
WHERE user_account.id > :id_1 RETURNING address.id, address.email_address


In [41]:
with engine.connect() as conn:
    result = conn.execute(insert_stmt)
    conn.commit()

2022-05-05 15:17:29,654 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:29,656 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || ? AS anon_1 
FROM user_account 
WHERE user_account.id > ?
2022-05-05 15:17:29,660 INFO sqlalchemy.engine.Engine [generated in 0.00542s] ('@mail.ru', 3)
2022-05-05 15:17:29,663 INFO sqlalchemy.engine.Engine COMMIT


Для более наглядной демонстрации примеров добавим некоторым пользователям адрес электронной почты в домене @yandex.ru

In [42]:
select_stmt = select(user_table.c.id, user_table.c.name + "@yandex.ru")
select_stmt = select_stmt.where(user_table.c.id > 4)

insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)

with engine.connect() as conn:
    result = conn.execute(insert_stmt)
    conn.commit()

2022-05-05 15:17:29,681 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:29,682 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || ? AS anon_1 
FROM user_account 
WHERE user_account.id > ?
2022-05-05 15:17:29,685 INFO sqlalchemy.engine.Engine [cached since 0.03028s ago] ('@yandex.ru', 4)
2022-05-05 15:17:29,687 INFO sqlalchemy.engine.Engine COMMIT


<p style="font-size:150%; font-weight:bold">4.2. Селекция строк с помощью Core или ORM</p>

И для Core, и для ORM функция *select()* генерирует конструкцию Select, которая используется для всех запросов SELECT. Передаваемый таким методам, как `Connection.execute()` в Core и `Session.execute()` в ORM, оператор SELECT выполняется в текущей транзакции, а строки результатов доступны через возвращаемый объект Result.


<p style="font-size:150%">4.2.1. Построение SQL-выражения с помощью метода select()</p>

Метод *select()* позволяет создать SQL-выражение аналогично тому, как это было рассмотрено выше для функции *insert()*.

In [43]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == 'tcampion')
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


Для выполнения SQL-выражения мы передаём его в исполняемый метод. Выражение с использованием команды SELECT возвращает строки, по которым можно совершить итерацию.

In [44]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-05-05 15:17:29,719 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:29,720 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-05-05 15:17:29,723 INFO sqlalchemy.engine.Engine [generated in 0.00390s] ('tcampion',)
(1, 'tcampion', 'Campion Thomas')
2022-05-05 15:17:29,727 INFO sqlalchemy.engine.Engine ROLLBACK


При использовании ORM мы передаём сформированное с помощью *select()* SQL-выражение методу `Session.execute()` в сеансе. Используя этот подход, мы продолжаем получать объекты *Row* из результата, однако эти строки теперь могут включать экземпляры класса User как отдельные элементы в каждой строке:

In [45]:
stmt = select(User).where(User.name == 'tcampion')
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-05-05 15:17:29,750 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:29,754 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-05-05 15:17:29,754 INFO sqlalchemy.engine.Engine [generated in 0.00222s] ('tcampion',)
(User(id=1, name='tcampion', fullname='Campion Thomas'),)
2022-05-05 15:17:29,766 INFO sqlalchemy.engine.Engine ROLLBACK


Распечатаем всех пользователей и все адреса пользователей.

In [46]:
stmt = select(User.id, User.name, User.fullname)
with Session(engine) as session:
    for row in session.execute(stmt):
        print(f"id:{row.id}, name:{row.name}, fullname:{row.fullname}")

2022-05-05 15:17:29,791 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:29,797 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-05-05 15:17:29,800 INFO sqlalchemy.engine.Engine [generated in 0.00214s] ()
id:1, name:tcampion, fullname:Campion Thomas
id:2, name:nnigrino, fullname:Nigrino Niccolo
id:3, name:gcaccini, fullname:Caccini Giulio
id:4, name:talbinoni, fullname:Albinoni Thomaso
id:5, name:fmilano, fullname:Da Milano Francesko
id:6, name:gbfontana, fullname:Giovanni Battista
2022-05-05 15:17:29,807 INFO sqlalchemy.engine.Engine ROLLBACK


In [47]:
stmt = select(Address.id, Address.user_id, Address.email_address)
with Session(engine) as session:
    for row in session.execute(stmt):
        print(f"id:{row.id}, user:{row.user_id}, email:{row.email_address}")

2022-05-05 15:17:29,830 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:29,830 INFO sqlalchemy.engine.Engine SELECT address.id, address.user_id, address.email_address 
FROM address
2022-05-05 15:17:29,839 INFO sqlalchemy.engine.Engine [generated in 0.00303s] ()
id:1, user:1, email:tcampion@aol.com
id:2, user:2, email:nnigrino@aol.com
id:3, user:3, email:gcaccini@aol.com
id:4, user:4, email:talbinoni@aol.com
id:5, user:5, email:fmilano@aol.com
id:6, user:6, email:gbfontana@aol.com
id:7, user:4, email:talbinoni@mail.ru
id:8, user:5, email:fmilano@mail.ru
id:9, user:6, email:gbfontana@mail.ru
id:10, user:5, email:fmilano@yandex.ru
id:11, user:6, email:gbfontana@yandex.ru
2022-05-05 15:17:29,848 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:150%">4.2.2. Определение предложений COLUMNS и FROM</p>

Функция *select()* принимает позиционные элементы, представляющие любое количество выражений *Column* и/или *Table*, а также широкий спектр совместимых объектов, которые преобразуются в список SQL-выражений. Эти элементы также служат  для создания предложения FROM, которое неявно выводится из переданных столбцов и табличных выражений.

In [48]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


Чтобы выбрать данные из отдельных столбцов с использованием подхода *Core*, доступ к объектам столбцов осуществляется с помощью `Table.c`. Предложение FROM будет сформировано как набор всех объектов *Table*.

In [49]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


<p style="font-size:125%">4.2.2.1. Выбор сущностей и столбцов с помощью ORM</p>

Выберем атрибуты класса *User*.

In [50]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


Далее, если выполнить это выражение с помощью метода `session.execute()`, мы получим объекты строк как 
экземпляры класса *User*.

In [51]:
row = session.execute(select(User)).first()
row

2022-05-05 15:17:29,934 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:29,942 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-05-05 15:17:29,942 INFO sqlalchemy.engine.Engine [generated in 0.00293s] ()


(User(id=1, name='tcampion', fullname='Campion Thomas'),)

Мы получили одну строку таблицы пользователей. К результату можно обратиться по индексу.

In [52]:
row[0]

User(id=1, name='tcampion', fullname='Campion Thomas')

Вместо выбора всех столбцов, мы можем указать отдельные атрибуты класса.

In [53]:
row = session.execute(select(User.name, User.fullname)).first()
row

2022-05-05 15:17:30,013 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2022-05-05 15:17:30,015 INFO sqlalchemy.engine.Engine [generated in 0.00194s] ()


('tcampion', 'Campion Thomas')

Покажем применение этой техники для выбора атрибутов из двух таблиц - пользователей и адресов.

In [54]:
session.execute(
    select(User.name, Address).
    where(User.id==Address.user_id).
    order_by(Address.id)
).all()

2022-05-05 15:17:30,041 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2022-05-05 15:17:30,049 INFO sqlalchemy.engine.Engine [generated in 0.00280s] ()


[('tcampion', Address(id=1, email_address='tcampion@aol.com')),
 ('nnigrino', Address(id=2, email_address='nnigrino@aol.com')),
 ('gcaccini', Address(id=3, email_address='gcaccini@aol.com')),
 ('talbinoni', Address(id=4, email_address='talbinoni@aol.com')),
 ('fmilano', Address(id=5, email_address='fmilano@aol.com')),
 ('gbfontana', Address(id=6, email_address='gbfontana@aol.com')),
 ('talbinoni', Address(id=7, email_address='talbinoni@mail.ru')),
 ('fmilano', Address(id=8, email_address='fmilano@mail.ru')),
 ('gbfontana', Address(id=9, email_address='gbfontana@mail.ru')),
 ('fmilano', Address(id=10, email_address='fmilano@yandex.ru')),
 ('gbfontana', Address(id=11, email_address='gbfontana@yandex.ru'))]

<p style="font-size:125%">4.2.2.2. Выбор строк с использованием метки</p>

Метод `ColumnElement.label()`, а также одноименный метод, доступный для атрибутов ORM, создаёт метку столбца или SQL-выражения, позволяя ему иметь конкретное имя в результирующем отношении. В дальнейшем эту метку можно использовать в коде программы.

In [55]:
from sqlalchemy import func, cast
stmt = (
    select(
        (('Username: ' + user_table.c.name + '\t').label('username'), \
         ('UserFullName: '+user_table.c.fullname).label('fullname')),
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row.username, row.fullname)

2022-05-05 15:17:30,082 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:30,091 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name || ? AS username, ? || user_account.fullname AS fullname 
FROM user_account ORDER BY user_account.name
2022-05-05 15:17:30,097 INFO sqlalchemy.engine.Engine [generated in 0.00936s] ('Username: ', '\t', 'UserFullName: ')
Username: fmilano	 UserFullName: Da Milano Francesko
Username: gbfontana	 UserFullName: Giovanni Battista
Username: gcaccini	 UserFullName: Caccini Giulio
Username: nnigrino	 UserFullName: Nigrino Niccolo
Username: talbinoni	 UserFullName: Albinoni Thomaso
Username: tcampion	 UserFullName: Campion Thomas
2022-05-05 15:17:30,101 INFO sqlalchemy.engine.Engine ROLLBACK


In [56]:
from sqlalchemy import text
stmt = (
    select(
        text("'текстовая константа'"), user_table.c.name
    ).where(user_table.c.id < 3).order_by(user_table.c.name)
)
with engine.connect() as conn:
    print(conn.execute(stmt).all())

2022-05-05 15:17:30,124 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:30,125 INFO sqlalchemy.engine.Engine SELECT 'текстовая константа', user_account.name 
FROM user_account 
WHERE user_account.id < ? ORDER BY user_account.name
2022-05-05 15:17:30,128 INFO sqlalchemy.engine.Engine [generated in 0.00369s] (3,)
[('текстовая константа', 'nnigrino'), ('текстовая константа', 'tcampion')]
2022-05-05 15:17:30,132 INFO sqlalchemy.engine.Engine ROLLBACK


Конструкция *text()* может использоваться для буквального ввода команд SQL. Мы можем получить больше функциональности из нашего текстового фрагмента, используя конструкцию *literal_column()*. Этот объект похож на *text()*, за исключением того, что вместо произвольного текста он явно представляет один «столбец», который затем может быть помечен и на него можно ссылаться в подзапросах и других выражениях:

In [57]:
from sqlalchemy import literal_column
stmt = (
    select(
        literal_column("'некоторый текст'").label("p"), user_table.c.name.label("q")
    ).order_by(user_table.c.name)
)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.p}, {row.q}")

2022-05-05 15:17:30,164 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:30,166 INFO sqlalchemy.engine.Engine SELECT 'некоторый текст' AS p, user_account.name AS q 
FROM user_account ORDER BY user_account.name
2022-05-05 15:17:30,167 INFO sqlalchemy.engine.Engine [generated in 0.00505s] ()
некоторый текст, fmilano
некоторый текст, gbfontana
некоторый текст, gcaccini
некоторый текст, nnigrino
некоторый текст, talbinoni
некоторый текст, tcampion
2022-05-05 15:17:30,175 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:150%">4.2.3. Предложение WHERE</p>

Предложение WHERE позволяет устанавливать условие (предикат) на отбор строк отношения. Используются операторы ==, !=, <, >, >=. 

In [58]:
clause_1 = (user_table.c.name == 'tcampion')
clause_2 = (address_table.c.user_id > 10)

print(select(user_table).where(clause_1))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


Для добавления условий, выполняющихся одновременно, метод *where()* можно повторять неограниченное число раз.

In [59]:
print(
    select(address_table.c.email_address).
    where(user_table.c.name == 'tcampion').
    where(address_table.c.user_id == user_table.c.id)
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


Также, один метод *where()* может содержать множество условий.

In [60]:
print(
    select(address_table.c.email_address).
    where(
         user_table.c.name == 'tcampion',
         address_table.c.user_id == user_table.c.id
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


Логическое сложение OR и умножение AND можно применить, если использовать <a href="https://docs.sqlalchemy.org/en/14/core/sqlelement.html#sqlalchemy.sql.expression"> функции </a> *or_()* и *and_()*.

In [61]:
# Получить почтовые адреса пользователей с именами vivaldin и bond.
from sqlalchemy import and_, or_
print(
    select(Address.email_address).
    where(
        and_(
            or_(User.name == 'tcampion', User.name == 'nnigrino'),
            Address.user_id == User.id
        )
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id


Для простых сравнений «на равенство» с одним объектом также есть популярный метод, известный как `Select.filter_by()`, который принимает аргументы ключевого слова, соответствующие ключам столбцов или именам атрибутов ORM.

In [62]:
print(
    select(User).filter_by(name='tcampion', fullname='Campion Thomas')
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1


См. также: <a href="https://docs.sqlalchemy.org/en/14/core/operators.html">использовнаие операторов в SQLAlchemy</a>

<p style="font-size:150%">4.2.4. Предложение FROM в явном виде и соединение JOIN</p>

Как мы видели ранее, предложение FROM обычно выводится на основе выражений, которые мы устанавливаем в предложении столбцов, а также других элементов Select. То есть, если мы устанавливаем один столбец из определенной таблицы в предложении COLUMNS, то эта таблица помещается в предложение FROM:

In [63]:
print(select(user_table.c.name))

SELECT user_account.name 
FROM user_account


Так же происходит, когда мы отбираем данные из столбцов, принадлежащих разным таблицам.

In [64]:
print(select(user_table.c.name, address_table.c.email_address))

SELECT user_account.name, address.email_address 
FROM user_account, address


Чтобы СОЕДИНИТЬ эти две таблицы вместе, мы обычно используем один из двух методов в Select. Первый — это метод `Select.join_from()`, который позволяет нам явно указать левую и правую стороны JOIN:

In [65]:
print(
    select(user_table.c.name, address_table.c.email_address).
    join_from(user_table, address_table)
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


Другим является метод `Select.join()`, который указывает только правую часть JOIN, левая сторона выводится неявно:

In [66]:
print(
    select(user_table.c.name, address_table.c.email_address).
    join(address_table)
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


В этих примерах условие соединения ON сформировано неявно. Однако, его можно указывать в явном виде.

У нас также есть возможность явно добавлять элементы в предложение FROM. Для этого в следующем примере мы используем метод `Select.select_from()` чтобы указать первую таблицу соединения *user_account* и далее `Select.join()`.

In [67]:
print(
    select(address_table.c.email_address).
    select_from(user_table).join(address_table)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


Другой пример, когда может понадобиться использование метода *select_from()* - использование функции *count()* по всем атрибутам без указания имен таблиц и атрибутов.

In [68]:
from sqlalchemy import func
print (
    select(func.count('*')).select_from(user_table)
)

SELECT count(:count_2) AS count_1 
FROM user_account


Больше примеров в <a href="https://docs.sqlalchemy.org/en/14/orm/queryguide.html">ORM Querying Guide</a>

<p style="font-size:125%">4.2.4.1. Формирование условия соединения</p>

Предыдущие примеры использования соединения JOIN показали, что конструкция Select может соединять две таблицы и автоматически создавать предложение ON. Так происходило, потому что таблица *address* содержит ограничение ForeignKeyConstraint, которое обеспечивает ссылку на таблицу *user_account*.

Если левая и правая стороны соединения не имеют такого ограничения или есть несколько ограничений, нам нужно указать предложение ON напрямую. В этом случае `Select.join()`, и `Select.join_from()` принимают дополнительный аргумент для предложения ON, который задается с использованием той же механики SQL-выражения, что мы видели в предложении WHERE:

In [69]:
print(
    select(address_table.c.email_address).
    select_from(user_table).
    join(address_table, user_table.c.id == address_table.c.user_id)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


<p style="font-size:125%">4.2.4.2. Внешнее и полное соединения</p>

Оба метода `Select.join()` и `Select.join_from()` принимают аргументы ключевых слов `Select.join.isouter` и `Select.join.full`, которые позволяют создавать предложения левого внешнего LEFT OUTER JOIN и полного FULL OUTER JOIN соединений:

In [70]:
print(
    select(user_table).join(address_table, isouter=True)
)

print(
    select(user_table).join(address_table, full=True)
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


Существуют также методы:

`Select.outerjoin()`, эквивалентный использованию `.join(..., isouter=True)`;<br>
`Select.outerjoin_from()`, эквивалентный использованию `join_from(..., isouter=True)`.

<Table align="left">
    <tr><td bgcolor=#adb5bd></td>
        <td style="text-align:left; font-size:10pt">
Примечание.

SQLAlchemy не поддерживает команду RIGHT OUTER JOIN. Вместо этого поменяйте местами таблицы и используйте LEFT OUTER JOIN.
        </td></tr>
</Table>

<p style="font-size:150%">4.2.5. Предложения ORDER BY, GROUP BY, HAVING</p>

<p style="font-size:120%">4.2.5.1. ORDER BY</p>

Для упорядочивания результатов запросов используется метод <a href="https://docs.sqlalchemy.org/en/14/core/selectable.html#sqlalchemy.sql.expression.Select.order_by">` Select.order_by() `.</a>

In [71]:
print(select(user_table).order_by(user_table.c.name))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.name


Для упорядочивания по возрастанию или убыванию использем методы `ColumnElement.asc()` или `ColumnElement.desc()`

In [72]:
print(select(User).order_by(User.fullname.desc()))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.fullname DESC


<p style="font-size:120%">4.2.5.2. Агрегатные функции с группировкой и фильтрацией</p>

Группировка в SQL позволяет вычислять значение агрегатной функции для группы строк. Агрегатная функция SQL выполняется с помощью специального конструктора *func.* Покажем это на примере вычисления количества пользователей в таблице *user_account*.

In [73]:
from sqlalchemy import func
count_fn = func.count(user_table.c.id)

with engine.connect() as conn:
    result = conn.execute(count_fn).scalar()
    print(result)

2022-05-05 15:17:30,578 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:30,581 INFO sqlalchemy.engine.Engine SELECT count(user_account.id) AS count_1 
FROM user_account
2022-05-05 15:17:30,581 INFO sqlalchemy.engine.Engine [generated in 0.00482s] ()
6
2022-05-05 15:17:30,581 INFO sqlalchemy.engine.Engine ROLLBACK


Более подробно о работе с функциями: https://docs.sqlalchemy.org/en/14/tutorial/data_select.html#tutorial-functions

HAVING позволяет применить фильтр к результатам группировки. Для этого используются методы `Select.group_by()` и `Select.having()`.

Покажем имена и количество адресов пользователей, имеющих более одного адреса.

In [74]:
with engine.connect() as conn:
    result = conn.execute(
        select(User.name, func.count(Address.id).label("count")).
        join(Address).
        group_by(User.name).
        having(func.count(Address.id) > 1)
    )
    print(result.all())

2022-05-05 15:17:30,612 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:30,614 INFO sqlalchemy.engine.Engine SELECT user_account.name, count(address.id) AS count 
FROM user_account JOIN address ON user_account.id = address.user_id GROUP BY user_account.name 
HAVING count(address.id) > ?
2022-05-05 15:17:30,615 INFO sqlalchemy.engine.Engine [generated in 0.00538s] (1,)
[('fmilano', 3), ('gbfontana', 3), ('talbinoni', 2)]
2022-05-05 15:17:30,615 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:120%">4.2.5.3. Использование метки для группировки и упорядочивания</p>

Иногда требуется группировать или упорядочивать по результатом выражений с участием столбцов. И для того, чтобы не вводить и не вычислять выражения заново, используются метки.

In [75]:
from sqlalchemy import func, desc
stmt = select(
        Address.user_id,
        func.count(Address.id).label('num_addresses')).\
        group_by('user_id').order_by(desc('num_addresses'), 'user_id')

with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
        print(f"{list(row.keys())[0]}: {row.user_id}, {list(row.keys())[1]}: {row.num_addresses}")

2022-05-05 15:17:30,642 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:30,646 INFO sqlalchemy.engine.Engine SELECT address.user_id, count(address.id) AS num_addresses 
FROM address GROUP BY address.user_id ORDER BY num_addresses DESC, address.user_id
2022-05-05 15:17:30,648 INFO sqlalchemy.engine.Engine [generated in 0.00368s] ()
user_id: 5, num_addresses: 3
user_id: 6, num_addresses: 3
user_id: 4, num_addresses: 2
user_id: 1, num_addresses: 1
user_id: 2, num_addresses: 1
user_id: 3, num_addresses: 1
2022-05-05 15:17:30,652 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:150%">4.2.6. Псевдонимы</p>

Псевдонимы используют для сокращённой записи наименований, а также чтобы избежать конфликтов и получить верный результат в случае, когда к одной и той же таблице в одном запросе обращаются более одного раза.

<p style="font-size:125%">4.2.6.1. Псевдонимы в интерфйсе Core</p>

В интерфейсе Core для псевдонима используется метод ` FromClause.alias()`. Покажем на примере выполнения запроса, в котором используются две таблицы *user_account* под разными псевдонимами. Найдём пары пользователей, идентификаторы которых отличаются в два раза.

In [76]:
usr_1 = user_table.alias('usr_1')
usr_2 = user_table.alias('usr_2')
stmt = select(usr_1.c.id, usr_1.c.name, usr_2.c.id, usr_2.c.name)
stmt = stmt.where(text('cast(usr_2.id as real)') / usr_1.c.id == 2.0)

with engine.connect() as conn:
    result = conn.execute(stmt).all()
    print(result)

2022-05-05 15:17:30,679 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:30,687 INFO sqlalchemy.engine.Engine SELECT usr_1.id, usr_1.name, usr_2.id AS id_1, usr_2.name AS name_1 
FROM user_account AS usr_1, user_account AS usr_2 
WHERE cast(usr_2.id as real) / usr_1.id = ?
2022-05-05 15:17:30,689 INFO sqlalchemy.engine.Engine [generated in 0.00427s] (2.0,)
[(1, 'tcampion', 2, 'nnigrino'), (2, 'nnigrino', 4, 'talbinoni'), (3, 'gcaccini', 6, 'gbfontana')]
2022-05-05 15:17:30,690 INFO sqlalchemy.engine.Engine ROLLBACK


C:\Users\myfri\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SAWarning: SELECT statement has a cartesian product between FROM element(s) "usr_1" and FROM element "usr_2".  Apply join condition(s) between each element to resolve.
  import sys


<p style="font-size:125%">4.2.6.2. Псевдонимы в интерфйсе ORM</p>

В интрефейсе ORM псевдонимы задаются с помощью функции *aliased()*. В следующем примере с помощью псевдонимов производится отбор пользователей, имеющих два определённых e-mail адреса.

In [77]:
from sqlalchemy.orm import aliased
address_alias_1 = aliased(Address)
address_alias_2 = aliased(Address)
stmt = (
    select(User).
    join_from(User, address_alias_1).\
    where(address_alias_1.email_address.like('%aol.com')).\
    join_from(User, address_alias_2).\
    where(address_alias_2.email_address.like('%@mail.ru'))
)
with engine.connect() as conn:
    result = conn.execute(stmt).all()
    print(result)

2022-05-05 15:17:30,726 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:30,729 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account JOIN address AS address_1 ON user_account.id = address_1.user_id JOIN address AS address_2 ON user_account.id = address_2.user_id 
WHERE address_1.email_address LIKE ? AND address_2.email_address LIKE ?
2022-05-05 15:17:30,729 INFO sqlalchemy.engine.Engine [generated in 0.00554s] ('%aol.com', '%@mail.ru')
[(4, 'talbinoni', 'Albinoni Thomaso'), (5, 'fmilano', 'Da Milano Francesko'), (6, 'gbfontana', 'Giovanni Battista')]
2022-05-05 15:17:30,729 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:150%">4.2.7. Подзапросы и табличные выражения (ТВ)</p>

Для определения подзапроса и ТВ в SQLAlchemy используются методы `Select.subquery()` и `Select.cte()`. Любой из таких объектов может использоваться в предложении FROM SQL-выражения. 

<p style="font-size:120%">4.2.7.1. Подзапросы</p>

Создадим выражение подзапроса для подсчёта числа адресов каждого пользователя. Используем метод `Select.subquery()`.

In [78]:
subq = select(
    func.count(address_table.c.id).label("count"),
    address_table.c.user_id
).group_by(address_table.c.user_id).subquery()

Сам по себе подзапрос может быть распечатан в текстовом виде без дополнительных скобок.

In [79]:
print(subq)

SELECT count(address.id) AS count, address.user_id 
FROM address GROUP BY address.user_id


Объект подзапроса в SQL-выражении ведёт себя также как и любой объект таблицы.

In [80]:
print(select(subq.c.user_id, subq.c.count))

SELECT anon_1.user_id, anon_1.count 
FROM (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1


Теперь мы можем связать подзапрос к таблице адресов с запросом *Select* к таблице пользователей.

In [81]:
stmt = select(
   user_table.c.name,
   user_table.c.fullname,
   subq.c.count
).join_from(user_table, subq)

print(stmt)

SELECT user_account.name, user_account.fullname, anon_1.count 
FROM user_account JOIN (SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id) AS anon_1 ON user_account.id = anon_1.user_id


В этом примере мы использовали метод `Select.join_from()` и неявное условие соединения `user_table.c.id==address_table.c.user_id`, обеспеченное ссылочной целостностью БД.

<p style="font-size:125%">4.2.7.2. Табличные выражения</p>

Использование табличных выражений (ТВ) аналогично подзапросам с той лишь разницей, что применяется метод метод `Select.cte()`. Однако, SQL-выражение в текстовом виде имеет больше отличий. Сравните:

In [82]:
subq = select(
    func.count(address_table.c.id).label('count'),
    address_table.c.user_id
).group_by(address_table.c.user_id).cte('A1')

stmt = select(
   user_table.c.name,
   user_table.c.fullname,
   subq.c.count
).join_from(user_table, subq)

print(stmt)

WITH "A1" AS 
(SELECT count(address.id) AS count, address.user_id AS user_id 
FROM address GROUP BY address.user_id)
 SELECT user_account.name, user_account.fullname, "A1".count 
FROM user_account JOIN "A1" ON user_account.id = "A1".user_id


<p style="font-size:125%">4.2.7.3. Подзапросы и табличные выражения в стиле ORM</p>
 
В интерфейсе ORM функция aliased() может быть использована для связывания ORM-сущностей, таких, как классы User или Address, с каким-либо источником строк, таким, как таблица или выборка с помощью SQL-запроса.

В следующем примере применим к подзапросу функцию <a href="https://docs.sqlalchemy.org/en/14/orm/query.html#sqlalchemy.orm.aliased">aliased()</a>, с помощью которой свяжем имя Address с записями таблицы адресов с учётом фильтра.

In [83]:
subq = select(Address).where(~Address.email_address.like('%@aol.com')).subquery()
address_subq = aliased(Address, subq)
stmt = select(User, address_subq)
stmt = stmt.join_from(User, address_subq)
stmt = stmt.order_by(User.id, address_subq.id)
with Session(engine) as session:
    for user, address in session.execute(stmt):
        print(f"{user} {address}")

2022-05-05 15:17:30,878 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:30,889 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname, anon_1.id AS id_1, anon_1.email_address, anon_1.user_id 
FROM user_account JOIN (SELECT address.id AS id, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE address.email_address NOT LIKE ?) AS anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.id
2022-05-05 15:17:30,892 INFO sqlalchemy.engine.Engine [generated in 0.00314s] ('%@aol.com',)
User(id=4, name='talbinoni', fullname='Albinoni Thomaso') Address(id=7, email_address='talbinoni@mail.ru')
User(id=5, name='fmilano', fullname='Da Milano Francesko') Address(id=8, email_address='fmilano@mail.ru')
User(id=5, name='fmilano', fullname='Da Milano Francesko') Address(id=10, email_address='fmilano@yandex.ru')
User(id=6, name='gbfontana', fullname='Giovanni Battista') Address(id=9, email_addre

Этот же пример с использованием ТВ.

In [84]:
cte_obj = select(Address).where(~Address.email_address.like('%@aol.com')).cte()
address_cte = aliased(Address, cte_obj)
stmt = select(User, address_cte).join_from(User, address_cte).order_by(User.id, address_cte.id)
with Session(engine) as session:
    for user, address in session.execute(stmt):
        print(f"{user} {address}")

2022-05-05 15:17:30,931 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:30,953 INFO sqlalchemy.engine.Engine WITH anon_1 AS 
(SELECT address.id AS id, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE address.email_address NOT LIKE ?)
 SELECT user_account.id, user_account.name, user_account.fullname, anon_1.id AS id_1, anon_1.email_address, anon_1.user_id 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.id
2022-05-05 15:17:30,953 INFO sqlalchemy.engine.Engine [generated in 0.00326s] ('%@aol.com',)
User(id=4, name='talbinoni', fullname='Albinoni Thomaso') Address(id=7, email_address='talbinoni@mail.ru')
User(id=5, name='fmilano', fullname='Da Milano Francesko') Address(id=8, email_address='fmilano@mail.ru')
User(id=5, name='fmilano', fullname='Da Milano Francesko') Address(id=10, email_address='fmilano@yandex.ru')
User(id=6, name='gbfontana', fullname='Giovanni Battista') Address(id=

<p style="font-size:150%">4.2.8. Скалярные и связанные подзапросы</p>

Скалярный подзапрос возвращает ноль или одну строку в одном столбце. Связанный подзапрос — это скалярный подзапрос, ссылающийся на таблицу внешнего запроса и возвращающий скаларное значение для каждой записи отношения, формируемого внешним запросом. Результат выполнения связанного скалярного запроса сказывается на значении предиката внешнего запроса. Скалярный подзапрос может использовать агрегатную функцию. Скалярный подзапрос необходимо обозначать явно с помощью метода `Select.scalar_subquery()`. В текстовом виде это обычное SQL-выражение, использующее команду SELECT с участием двух таблиц.

In [85]:
subq = select(func.count(address_table.c.id)).\
            where(user_table.c.id == address_table.c.user_id).\
            scalar_subquery()
print(subq)

(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id)


Объект подзапроса может участвовать в выражениях, например

In [86]:
print(subq == 5)

(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id) = :param_1


Если подзапрос к таблице *address* указать в конструкторе *Select*, а именно в перечислении столбцов вместе со столбцами таблицы *user_account*, призойдёт автоматическое связывание *user_account* с *address*, при этом таблица *user_account* не появляется в предложении FROM основного запроса.

In [87]:
stmt = select(user_table.c.name, subq.label("address_count"))
print(stmt)

SELECT user_account.name, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account


Возникает ошибка, если связывание неоднозначно. В следующем примере одним из источников строк является подзапрос *subq*. Этот источник может быть связан как с *user_table*, так и с *address_table*. Поэтому возникает ошибка в ходе формирования предложения FROM SQL-выражения.

```
stmt = select(
    user_table.c.name,
    address_table.c.email_address,
    subq.label("address_count")
).\
join_from(user_table, address_table).\
order_by(user_table.c.id, address_table.c.id)
print(stmt)

...
InvalidRequestError: Select statement '<... Select object at ...>' returned
no FROM clauses due to auto-correlation; specify correlate(<tables>) to
control correlation manually.
```

Чтобы избежать ошибки, требуется в явном виде указать, что подзапрос *subq* связывается с таблицей *user_table*. Для связывания используем метод <a href="https://docs.sqlalchemy.org/en/14/core/selectable.html#sqlalchemy.sql.expression.ScalarSelect.correlate"> `ScalarSelect.correlate()`</a> или метод <a href="https://docs.sqlalchemy.org/en/14/core/selectable.html#sqlalchemy.sql.expression.ScalarSelect.correlate_except">ScalarSelect.correlate_except()</a>

In [88]:
subq = select(func.count(address_table.c.id)).\
            where(user_table.c.id == address_table.c.user_id).\
            scalar_subquery().correlate(user_table)
print(subq)

(SELECT count(address.id) AS count_1 
FROM address, user_account 
WHERE user_account.id = address.user_id)


Теперь используем этот подзапрос.

In [89]:
with engine.connect() as conn:
    result = conn.execute(
        select(
            user_table.c.name,
            address_table.c.email_address,
            subq.label("address_count")
            ).
        join_from(user_table, address_table).
        order_by(user_table.c.id, address_table.c.id)
    )
    for item in result:
        print(item)

2022-05-05 15:17:31,068 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,073 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.email_address, (SELECT count(address.id) AS count_1 
FROM address 
WHERE user_account.id = address.user_id) AS address_count 
FROM user_account JOIN address ON user_account.id = address.user_id ORDER BY user_account.id, address.id
2022-05-05 15:17:31,073 INFO sqlalchemy.engine.Engine [generated in 0.00532s] ()
('tcampion', 'tcampion@aol.com', 1)
('nnigrino', 'nnigrino@aol.com', 1)
('gcaccini', 'gcaccini@aol.com', 1)
('talbinoni', 'talbinoni@aol.com', 2)
('talbinoni', 'talbinoni@mail.ru', 2)
('fmilano', 'fmilano@aol.com', 3)
('fmilano', 'fmilano@mail.ru', 3)
('fmilano', 'fmilano@yandex.ru', 3)
('gbfontana', 'gbfontana@aol.com', 3)
('gbfontana', 'gbfontana@mail.ru', 3)
('gbfontana', 'gbfontana@yandex.ru', 3)
2022-05-05 15:17:31,081 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:150%">4.2.9. Объединение и операции над множествами</p>

Рассмотрим работу операции объединения с помощью UNION, а также операции над множествами с помощью INTERSECT [ALL] и EXCEPT [ALL]. Для выполнения этих операций в библиотеке SQLAlchemy используются методы `union_all()`, `intersect_all()` и `except_all()`.

In [90]:
from sqlalchemy import union_all

stmt1 = select(user_table).where(user_table.c.name == 'tcampion')
stmt2 = select(user_table).where(user_table.c.name == 'gcaccini')
u = union_all(stmt1, stmt2)

with engine.connect() as conn:
    result = conn.execute(u)
    print(result.all())

2022-05-05 15:17:31,114 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,116 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-05-05 15:17:31,116 INFO sqlalchemy.engine.Engine [generated in 0.00400s] ('tcampion', 'gcaccini')
[(1, 'tcampion', 'Campion Thomas'), (3, 'gcaccini', 'Caccini Giulio')]
2022-05-05 15:17:31,116 INFO sqlalchemy.engine.Engine ROLLBACK


Результат объединения можно использовать в подзапросе. Для этого в следующем примере с помощью метода `SelectBase.subquery()` создадим объект подзапроса с коллекцией <a href="https://docs.sqlalchemy.org/en/14/core/selectable.html#sqlalchemy.sql.expression.FromClause.c">`FromClause.c`</a>, на элементы которой можно в дальнейшем ссылаться во внешнем запросе. В следующем примере к именам пользователей из результата объединения добавим информацию об адресах.

In [91]:
u_subq = u.subquery()
stmt = (
    select(u_subq.c.name, address_table.c.email_address).
    join_from(address_table, u_subq).
    order_by(u_subq.c.name, address_table.c.email_address)
)
with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-05-05 15:17:31,152 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,157 INFO sqlalchemy.engine.Engine SELECT anon_1.name, address.email_address 
FROM address JOIN (SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ?) AS anon_1 ON anon_1.id = address.user_id ORDER BY anon_1.name, address.email_address
2022-05-05 15:17:31,157 INFO sqlalchemy.engine.Engine [generated in 0.00549s] ('tcampion', 'gcaccini')
[('gcaccini', 'gcaccini@aol.com'), ('tcampion', 'tcampion@aol.com')]
2022-05-05 15:17:31,157 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:125%">4.2.9.1. Объединение с помощью ORM</p>

Составим объединение результатов двух выборок.

In [92]:
stmt1 = select(User).where(User.name == 'tcampion')
stmt2 = select(User).where(User.name == 'gcaccini')
u = union_all(stmt1, stmt2)

with engine.connect() as conn:
    result = conn.execute(stmt).all()
    print(result)

2022-05-05 15:17:31,195 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,197 INFO sqlalchemy.engine.Engine SELECT anon_1.name, address.email_address 
FROM address JOIN (SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ?) AS anon_1 ON anon_1.id = address.user_id ORDER BY anon_1.name, address.email_address
2022-05-05 15:17:31,197 INFO sqlalchemy.engine.Engine [cached since 0.04716s ago] ('tcampion', 'gcaccini')
[('gcaccini', 'gcaccini@aol.com'), ('tcampion', 'tcampion@aol.com')]
2022-05-05 15:17:31,205 INFO sqlalchemy.engine.Engine ROLLBACK


Аналогичный результат получим с помощью методов `.from_statement()` и `.scalars()`.

In [93]:
orm_stmt = select(User).from_statement(u)
with Session(engine) as session:
    for obj in session.execute(orm_stmt).scalars():
        print(obj)

2022-05-05 15:17:31,230 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,231 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-05-05 15:17:31,231 INFO sqlalchemy.engine.Engine [generated in 0.00294s] ('tcampion', 'gcaccini')
User(id=1, name='tcampion', fullname='Campion Thomas')
User(id=3, name='gcaccini', fullname='Caccini Giulio')
2022-05-05 15:17:31,239 INFO sqlalchemy.engine.Engine ROLLBACK


Результат объединения также можно использовать в качестве подзапроса в стиле ORM. Следующий пример даёт такй же результат объединения, как и представленный выше.

In [94]:
user_alias = aliased(User, u.subquery())
orm_stmt = select(user_alias).order_by(user_alias.id)
with Session(engine) as session:
    for obj in session.execute(orm_stmt).scalars():
        print(obj)

2022-05-05 15:17:31,302 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,317 INFO sqlalchemy.engine.Engine SELECT anon_1.id, anon_1.name, anon_1.fullname 
FROM (SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ? UNION ALL SELECT user_account.id AS id, user_account.name AS name, user_account.fullname AS fullname 
FROM user_account 
WHERE user_account.name = ?) AS anon_1 ORDER BY anon_1.id
2022-05-05 15:17:31,321 INFO sqlalchemy.engine.Engine [generated in 0.00261s] ('tcampion', 'gcaccini')
User(id=1, name='tcampion', fullname='Campion Thomas')
User(id=3, name='gcaccini', fullname='Caccini Giulio')
2022-05-05 15:17:31,322 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:150%">4.2.10. Подзапросы существования EXISTS</p>

Функция EXISTS языка SQL используется в скалярных подзапросах, возвращающих логическое значение. В следующем примере с помощью метода `SelectBase.exists()` найдём пользователей, имеющих более одного адреса.

In [95]:
address_table_2 = address_table.alias('address_2')

subq = (
    select(address_table_2.c.id)\
    .where(address_table_2.c.user_id == user_table.c.id)\
    .where(address_table_2.c.id != address_table.c.id)\
).exists()
with engine.connect() as conn:
    result = conn.execute(
        select(user_table.c.name).distinct().join(address_table).where(subq)
    )
    print(result.all())

2022-05-05 15:17:31,364 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,370 INFO sqlalchemy.engine.Engine SELECT DISTINCT user_account.name 
FROM user_account JOIN address ON user_account.id = address.user_id 
WHERE EXISTS (SELECT address_2.id 
FROM address AS address_2 
WHERE address_2.user_id = user_account.id AND address_2.id != address.id)
2022-05-05 15:17:31,371 INFO sqlalchemy.engine.Engine [generated in 0.00586s] ()
[('talbinoni',), ('fmilano',), ('gbfontana',)]
2022-05-05 15:17:31,371 INFO sqlalchemy.engine.Engine ROLLBACK


Подзапросы существования часто используют с отрицанием. Найдём пользователей, у которых нет адресов в домене @mail.ru

In [96]:
subq = (
    select(address_table.c.id).
    where(user_table.c.id == address_table.c.user_id).
    where(address_table.c.email_address.like("%@mail.ru"))
).exists()
with engine.connect() as conn:
    result = conn.execute(
        select(user_table.c.name).where(~subq)
    )
    print(result.all())

2022-05-05 15:17:31,403 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,406 INFO sqlalchemy.engine.Engine SELECT user_account.name 
FROM user_account 
WHERE NOT (EXISTS (SELECT address.id 
FROM address 
WHERE user_account.id = address.user_id AND address.email_address LIKE ?))
2022-05-05 15:17:31,407 INFO sqlalchemy.engine.Engine [generated in 0.00405s] ('%@mail.ru',)
[('tcampion',), ('nnigrino',), ('gcaccini',)]
2022-05-05 15:17:31,412 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:150%">4.2.11. Функции SQL</p>

Ранее мы уже использовали конструктор <a href="https://docs.sqlalchemy.org/en/14/core/sqlelement.html#sqlalchemy.sql.expression.func">func</a> для создания объектов функций. Приведем несколько примеров использования наиболее часто встречающихся функций.

- функция count(). Агрегатная функция, возвращающая число строк.

In [97]:
print(select(func.count()).select_from(user_table))

SELECT count(*) AS count_1 
FROM user_account


- функция lower(). текстовая функция, возвращающая строку строчными буквами.

In [98]:
print(select(func.lower("A String With Much UPPERCASE")))

SELECT lower(:lower_2) AS lower_1


- Функция now(). Возвращает текущую дату и время.

In [99]:
stmt = select(func.now())
with engine.connect() as conn:
    result = conn.execute(stmt)
    print(result.all())

2022-05-05 15:17:31,477 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,479 INFO sqlalchemy.engine.Engine SELECT CURRENT_TIMESTAMP AS now_1
2022-05-05 15:17:31,481 INFO sqlalchemy.engine.Engine [generated in 0.00379s] ()
[(datetime.datetime(2022, 5, 5, 12, 17, 31),)]
2022-05-05 15:17:31,483 INFO sqlalchemy.engine.Engine ROLLBACK


Необходимо помнить, что особенности использования функций зависят от конкретной СУБД.

In [100]:
from sqlalchemy.dialects import postgresql
print(select(func.now()).compile(dialect=postgresql.dialect()))

from sqlalchemy.dialects import oracle
print(select(func.now()).compile(dialect=oracle.dialect()))

SELECT now() AS now_1
SELECT CURRENT_TIMESTAMP AS now_1 FROM DUAL


<p style="font-size:125%">4.2.11.1. Возвращаемый тип данных SQL-функции</p>

Поскольку функции участвуют в выражениях со значениями столбцов, они также имеют типы данных SQL, которые описывают тип данных сгенерированного выражения SQL.

Доступ к возвращаемому типу SQL-выржения любой функции SQL можно получить, как правило, в целях отладки, обратившись к атрибуту `Function.type`:

In [101]:
print(func.now().type)

s = select(func.now().label('now'))
with engine.connect() as conn:
    result = conn.execute(s).scalars()
    for row in result:
        print('What time is now:', row)

DATETIME
2022-05-05 15:17:31,596 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,597 INFO sqlalchemy.engine.Engine SELECT CURRENT_TIMESTAMP AS now
2022-05-05 15:17:31,599 INFO sqlalchemy.engine.Engine [generated in 0.00348s] ()
What time is now: 2022-05-05 12:17:31
2022-05-05 15:17:31,602 INFO sqlalchemy.engine.Engine ROLLBACK


Используем функцию SQLite *json_object()* для вывода результатов запроса в формате JSON.

In [102]:
f = func.json_object('id', user_table.c.id,\
                    'name', user_table.c.name,\
                    'address', address_table.c.email_address)

stmt = select(f)
stmt = stmt.join_from(user_table, address_table)
stmt = stmt.where(user_table.c.id < 3)

print(stmt)

"""
with engine.connect() as conn:
    result = conn.execute(stmt).all()
    for row in result:
        print(row[0])"""

SELECT json_object(:json_object_2, user_account.id, :json_object_3, user_account.name, :json_object_4, address.email_address) AS json_object_1 
FROM user_account JOIN address ON user_account.id = address.user_id 
WHERE user_account.id < :id_1


'\nwith engine.connect() as conn:\n    result = conn.execute(stmt).all()\n    for row in result:\n        print(row[0])'

<p style="font-size:125%">4.2.11.2. Оконные функции</p>

Оконная функция позволяет использовать агрегатные функции SQL особым образом, вычисляя совокупное значение по строкам, возвращаемым в группе, по мере обработки отдельных строк результата. В то время как агрегатная функция, такая как *max()*, даст максимальное значение столбца в наборе строк, использование такой же оконной функции позволяет получить максимальное значение для каждой строки.

В SQL оконные функции позволяют указать строки, к которым функция должна применяться, значение «подраздела», которое рассматривает окно по различным подмножествам строк, и выражение «упорядочить по», которое указывает порядок, в котором оконная функция применяется к строкам.

В SQLAlchemy все функции SQL, сгенерированные пространством имен func, включают метод `FunctionElement.over()`, который предоставляет оконную функцию или синтаксис «OVER»; полученная конструкция является конструкцией Over.

Обычной функцией, используемой с оконными функциями, является функция row_number(), которая просто подсчитывает строки. Мы можем разделить это количество строк по имени пользователя, чтобы пронумеровать адреса электронной почты отдельных пользователей. В следующем примере разделим все адреса на подгруппы по имени пользователя и отсортируем в обратном порядке по адресу.

In [103]:
stmt = select(
    func.row_number().over(partition_by=user_table.c.name, \
                           order_by=desc(address_table.c.email_address)),
    user_table.c.name,
    address_table.c.email_address
).select_from(user_table).join(address_table)
with engine.connect() as conn:  
    result = conn.execute(stmt)
    for row in result:
        print(row)

2022-05-05 15:17:31,678 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,680 INFO sqlalchemy.engine.Engine SELECT row_number() OVER (PARTITION BY user_account.name ORDER BY address.email_address DESC) AS anon_1, user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
2022-05-05 15:17:31,683 INFO sqlalchemy.engine.Engine [generated in 0.00443s] ()
(1, 'fmilano', 'fmilano@yandex.ru')
(2, 'fmilano', 'fmilano@mail.ru')
(3, 'fmilano', 'fmilano@aol.com')
(1, 'gbfontana', 'gbfontana@yandex.ru')
(2, 'gbfontana', 'gbfontana@mail.ru')
(3, 'gbfontana', 'gbfontana@aol.com')
(1, 'gcaccini', 'gcaccini@aol.com')
(1, 'nnigrino', 'nnigrino@aol.com')
(1, 'talbinoni', 'talbinoni@mail.ru')
(2, 'talbinoni', 'talbinoni@aol.com')
(1, 'tcampion', 'tcampion@aol.com')
2022-05-05 15:17:31,702 INFO sqlalchemy.engine.Engine ROLLBACK


В следующем примере мы применили три оконных функции. Первая функция возвращает порядковый номер строки, вторая функция возвращает число строк для одинаковых имен пользователей с накоплением, последняя функция ыозвращает количество строк результата.

In [104]:
stmt = select(
    func.row_number().over(),
    func.count().over(order_by=user_table.c.name),
    func.count().over(),
    user_table.c.name,
    address_table.c.email_address).select_from(user_table).join(address_table)

with engine.connect() as conn:  
    result = conn.execute(stmt)
    for row in result:
        print(row)

2022-05-05 15:17:31,737 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,740 INFO sqlalchemy.engine.Engine SELECT row_number() OVER () AS anon_1, count(*) OVER (ORDER BY user_account.name) AS anon_2, count(*) OVER () AS anon_3, user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
2022-05-05 15:17:31,744 INFO sqlalchemy.engine.Engine [generated in 0.00714s] ()
(1, 3, 11, 'fmilano', 'fmilano@aol.com')
(2, 3, 11, 'fmilano', 'fmilano@mail.ru')
(3, 3, 11, 'fmilano', 'fmilano@yandex.ru')
(4, 6, 11, 'gbfontana', 'gbfontana@aol.com')
(5, 6, 11, 'gbfontana', 'gbfontana@mail.ru')
(6, 6, 11, 'gbfontana', 'gbfontana@yandex.ru')
(7, 7, 11, 'gcaccini', 'gcaccini@aol.com')
(8, 8, 11, 'nnigrino', 'nnigrino@aol.com')
(9, 10, 11, 'talbinoni', 'talbinoni@aol.com')
(10, 10, 11, 'talbinoni', 'talbinoni@mail.ru')
(11, 11, 11, 'tcampion', 'tcampion@aol.com')
2022-05-05 15:17:31,751 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:125%">4.2.11.3. Предложение FILTER</p>

Многие СУБД поддерживают предложение FILTER для ограничения числа строк, на которые действует агрегатная функция.

In [105]:
stmt = select(
    func.count(address_table.c.email_address).filter(user_table.c.name == 'fmilano'),
    func.count(address_table.c.email_address).filter(user_table.c.name == 'talbinoni')
).select_from(user_table).join(address_table)
with engine.connect() as conn:  
    result = conn.execute(stmt)
    print(result.all())

2022-05-05 15:17:31,778 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,784 INFO sqlalchemy.engine.Engine SELECT count(address.email_address) FILTER (WHERE user_account.name = ?) AS anon_1, count(address.email_address) FILTER (WHERE user_account.name = ?) AS anon_2 
FROM user_account JOIN address ON user_account.id = address.user_id
2022-05-05 15:17:31,787 INFO sqlalchemy.engine.Engine [generated in 0.00384s] ('fmilano', 'talbinoni')
[(3, 2)]
2022-05-05 15:17:31,789 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:150%; font-weight:bold">4.3. Обновление и удаление строк с помощью Core</p>

В этом разделе рассмотрим возможности интерфейса Core по обновлению и удалению данных.

<p style="font-size:150%">4.3.1. Обновление</p>

Как правило, команда UPDATE языка SQL одновляет данные в одной таблице по условию, при этом ничего не возвращает. Однако, в некоторых СУБД релизована команда UPDATE, способная обновлять сразу несколько таблиц. Кроме того, с помощью предложения RETURNING некоторые СУБД пвозвращают обновлённые данные. Стандартная команда UPDATE применяется с помощью Core следующим образом:

In [106]:
from sqlalchemy import update
stmt = (
    update(user_table).where(user_table.c.name == 'talbinoni').
    values(name = 'rgiazotto', fullname='Giazotto Remo')
)
print(stmt, stmt.compile().params)

UPDATE user_account SET name=:name, fullname=:fullname WHERE user_account.name = :name_1 {'name': 'rgiazotto', 'fullname': 'Giazotto Remo', 'name_1': 'talbinoni'}


Метод `Update.values()` управляет содержимым предложения SET команды UPDATE. Похожий метод `Insert.values()` ранее применялся нами для вставки записей. Обновляемые значения передаются в функцию с помощь имен столбцов, выступающих здесь в роли именованных  аргументов.

SQLAlchemy поддерживает все основные формы команды UPDATE, в том числе, позволяет использовать SQL-выражения с участием столбцов.

In [107]:
user_table

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [108]:
stmt = (
    update(user_table).
    values(name="Username: " + user_table.c.name)
)
print(stmt)

UPDATE user_account SET name=(:name_1 || user_account.name)


Для выполнения команды UPDATE в стиле executemany() используется функция `bindparam()`.

In [109]:
from sqlalchemy import bindparam
stmt = (
  update(user_table).
  where(user_table.c.name == bindparam('oldname')).
  values(name=bindparam('newname'))
)
with engine.begin() as conn:
  conn.execute(
      stmt,
      [
         {'oldname':'jack', 'newname':'ed'},
         {'oldname':'wendy', 'newname':'mary'},
         {'oldname':'jim', 'newname':'jake'},
      ]
  )

2022-05-05 15:17:31,890 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:31,894 INFO sqlalchemy.engine.Engine UPDATE user_account SET name=? WHERE user_account.name = ?
2022-05-05 15:17:31,900 INFO sqlalchemy.engine.Engine [generated in 0.00279s] (('ed', 'jack'), ('mary', 'wendy'), ('jake', 'jim'))
2022-05-05 15:17:31,902 INFO sqlalchemy.engine.Engine COMMIT


<p style="font-size:125%">Связанное обновление</p>

Команда UPDATE может использовать строки в других таблицах с помощью связанного подзапроса.

In [110]:
scalar_subq = (
  select(address_table.c.email_address).
  where(address_table.c.user_id == user_table.c.id).
  order_by(address_table.c.id).
  limit(1).
  scalar_subquery()
)
update_stmt = update(user_table).values(fullname=scalar_subq)
print(update_stmt)

UPDATE user_account SET fullname=(SELECT address.email_address 
FROM address 
WHERE address.user_id = user_account.id ORDER BY address.id
 LIMIT :param_1)


<p style="font-size:125%">UPDATE ..FROM</p>

Некоторые СУБД, такие, как PostgreSQL и MySQL поддерживают синтаксис "UPDATE FROM", который позволяет указывать дополнительные таблицы в предложении FROM. Такой синтасис будет сгенерирован неявно в случае, когда в предложении WHERE SQL-выражения появляются дполнительные таблицы.

In [111]:
update_stmt = (
   update(user_table).
   where(user_table.c.id == address_table.c.user_id).
   where(address_table.c.email_address == 'patrick@aol.com').
   values(fullname='Pat')
 )
print(update_stmt)

UPDATE user_account SET fullname=:fullname FROM address WHERE user_account.id = address.user_id AND address.email_address = :email_address_1


<p style="font-size:125%">Параметризованное упорядоченное обновление</p>

Другой особенностью MySQL является обновление с учетом порядка параметров в предложении SET. Для этой цели в SQLAlchemy предусмотрен метод `Update.ordered_values()`, принимающий последовательность кортежей.

In [112]:
update_stmt = (
    update(some_table).
    ordered_values(
        (some_table.c.y, 20),
        (some_table.c.x, some_table.c.y + 10)
    )
)
print(update_stmt)

UPDATE some_table SET y=:y, x=(some_table.y + :y_1)


<p style="font-size:150%">4.3.2. Удаление</p>

Для удалния строк в SQLAlchemy предусмотрена функция delete(). Как правило, данная функция не возвращает удаленные строки, если не применяется поддерживаемое некоторыми СУБД предложение RETURNING.

In [113]:
from sqlalchemy import delete
stmt = delete(user_table).where(user_table.c.name == 'patrick')
print(stmt)

DELETE FROM user_account WHERE user_account.name = :name_1


<p style="font-size:125%">Удаление с помощью нескольких таблиц</p>

Функция delete(), так же, как и update(), может использовать несколько таблиц. данная функциональность является особенностью СУБД MySQL.

In [114]:
delete_stmt = (
   delete(user_table).
   where(user_table.c.id == address_table.c.user_id).
   where(address_table.c.email_address == 'patrick@aol.com')
 )
from sqlalchemy.dialects import mysql
print(delete_stmt.compile(dialect=mysql.dialect()))

DELETE FROM user_account USING user_account, address WHERE user_account.id = address.user_id AND address.email_address = %s


<p style="font-size:150%">4.3.3. Получение числа удалённых или обновлённых строк</p>

In [115]:
stmt = (
    update(user_table).where(user_table.c.name == 'talbinoni').
    values(name = 'rgiazotto', fullname='Giazotto Remo')
)
with engine.begin() as conn:
    result = conn.execute(stmt)
    print(result.rowcount)

2022-05-05 15:17:32,093 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-05 15:17:32,093 INFO sqlalchemy.engine.Engine UPDATE user_account SET name=?, fullname=? WHERE user_account.name = ?
2022-05-05 15:17:32,101 INFO sqlalchemy.engine.Engine [generated in 0.00243s] ('rgiazotto', 'Giazotto Remo', 'talbinoni')
1
2022-05-05 15:17:32,109 INFO sqlalchemy.engine.Engine COMMIT


<p style="font-size:150%">4.3.4. Предложение RETURNING в командах UPDATE и DELETE</p>

Поддержка предложения RETURNING реализована с помощью методов `Update.returning()` и `Delete.returning()`. Если применить эти методы в запросах к СУБД, поддерживающей RETURNING, они вернут затронутые изменениями записи в итерируемом объекте Result.

In [116]:
update_stmt = (
    update(user_table).where(user_table.c.name == 'patrick').
    values(fullname='Patrick the Star').
    returning(user_table.c.id, user_table.c.name)
)
print(update_stmt)


delete_stmt = (
    delete(user_table).where(user_table.c.name == 'patrick').
    returning(user_table.c.id, user_table.c.name)
)
print(delete_stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1 RETURNING user_account.id, user_account.name
DELETE FROM user_account WHERE user_account.name = :name_1 RETURNING user_account.id, user_account.name


<p style="font-size:150%; font-weight:bold">Источники</p>

Дополнительные источники информации о реализации команд UPDATE и DELETE с помощью SQLAlchemy.

- <a href="https://docs.sqlalchemy.org/en/14/core/dml.html#sqlalchemy.sql.expression.Update">Update</a><br>
- <a href="https://docs.sqlalchemy.org/en/14/core/dml.html#sqlalchemy.sql.expression.Delete">Delete</a><br>
- <a href="https://docs.sqlalchemy.org/en/14/tutorial/orm_data_manipulation.html#tutorial-orm-enabled-update">ORM-enabled UPDATE statements</a><br>
- <a href="https://docs.sqlalchemy.org/en/14/tutorial/orm_data_manipulation.html#tutorial-orm-enabled-delete">ORM-enabled DELETE Statements</a><br>

[К оглавлению](#Home)

<a id=5></a>

# Манипуляция данными с помощью ORM

<p style="font-size:150%">Вставка строк с помощью ORM</p>

При использовании ORM объект Сеанса отвечает за создание конструкций Вставки и их отправку в транзакции. Способ, которым мы инструктируем Сеанс сделать это, заключается в добавлении к нему записей объекта; Затем Сеанс гарантирует, что эти новые записи будут отправлены в базу данных, когда это потребуется, используя процесс, известный как сброс.


<p style="font-size:125%">Экземпляры классов представляют строки</p>

В то время как в предыдущем примере мы сгенерировали вставку, используя словари Python, чтобы указать данные, которые мы хотели добавить, в случае применения интерфейса ORM мы напрямую используем пользовательские классы Python, которые мы определили ранее в разделе «Определение метаданных таблицы с помощью ORM». Классы User и Address служили местом для определения того, как должны выглядеть соответствующие таблицы базы данных. Эти классы также служат расширяемыми объектами данных, которые мы также используем для создания и управления строками внутри транзакции. Ниже мы создадим два объекта User, каждый из которых представляет потенциальную строку базы данных для вставки:

In [117]:
squidward = User(name="squidward", fullname="Squidward Tentacles")
krabs = User(name="ehkrabs", fullname="Eugene H. Krabs")

Мы можем создавать эти объекты, используя имена сопоставленных столбцов в качестве аргументов ключевого слова в конструкторе. Это возможно, поскольку класс User включает в себя автоматически сгенерированный конструктор `__init__()`, который был предоставлен сопоставлением ORM, чтобы мы могли создавать каждый объект, используя имена столбцов в качестве ключей в конструкторе.

In [118]:
squidward

User(id=None, name='squidward', fullname='Squidward Tentacles')